# Gawain sa Pag-uuri ng Teksto

Tulad ng nabanggit, magtutuon tayo sa isang simpleng gawain sa pag-uuri ng teksto gamit ang **AG_NEWS** dataset, kung saan kailangang uriin ang mga headline ng balita sa isa sa 4 na kategorya: World, Sports, Business, at Sci/Tech.

## Ang Dataset

Ang dataset na ito ay naka-built-in sa [`torchtext`](https://github.com/pytorch/text) module, kaya madali natin itong ma-access.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Narito, ang `train_dataset` at `test_dataset` ay naglalaman ng mga koleksyon na nagbabalik ng mga pares ng label (numero ng klase) at teksto, halimbawa:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Kaya, i-print natin ang unang 10 bagong headline mula sa ating dataset:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Dahil ang mga dataset ay mga iterator, kung nais nating gamitin ang data nang maraming beses kailangan natin itong i-convert sa listahan:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenisasyon

Ngayon, kailangan nating i-convert ang teksto sa **mga numero** na maaaring i-representa bilang mga tensor. Kung nais natin ng word-level na representasyon, kailangan nating gawin ang dalawang bagay:
* gumamit ng **tokenizer** upang hatiin ang teksto sa **mga token**
* bumuo ng isang **bokabularyo** ng mga token na iyon.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Gamit ang bokabularyo, madali nating ma-encode ang ating tokenized na string sa isang set ng mga numero:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Bag of Words na representasyon ng teksto

Dahil ang mga salita ay may kahulugan, minsan ay maaari nating maunawaan ang kahulugan ng isang teksto sa pamamagitan lamang ng pagtingin sa mga indibidwal na salita, kahit na hindi natin isaalang-alang ang pagkakasunod-sunod ng mga ito sa pangungusap. Halimbawa, kapag nag-uuri ng balita, ang mga salitang tulad ng *weather*, *snow* ay malamang na tumutukoy sa *weather forecast*, habang ang mga salitang tulad ng *stocks*, *dollar* ay maaaring magpahiwatig ng *financial news*.

Ang **Bag of Words** (BoW) na representasyon ng vector ay ang pinakakaraniwang ginagamit na tradisyunal na representasyon ng vector. Ang bawat salita ay naka-link sa isang index ng vector, at ang elemento ng vector ay naglalaman ng bilang ng paglitaw ng isang salita sa isang partikular na dokumento.

![Larawan na nagpapakita kung paano kinakatawan ang bag of words na representasyon ng vector sa memorya.](../../../../../translated_images/tl/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Maaari mo ring isipin ang BoW bilang kabuuan ng lahat ng one-hot-encoded na mga vector para sa bawat indibidwal na salita sa teksto.

Narito ang isang halimbawa kung paano bumuo ng isang bag of words na representasyon gamit ang Scikit Learn python library:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Upang makalkula ang bag-of-words na vector mula sa vector na representasyon ng ating AG_NEWS dataset, maaari nating gamitin ang sumusunod na function:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Tandaan:** Dito, ginagamit natin ang global na variable na `vocab_size` upang tukuyin ang default na laki ng bokabularyo. Dahil kadalasan ang laki ng bokabularyo ay medyo malaki, maaari nating limitahan ang laki ng bokabularyo sa mga pinakakaraniwang salita. Subukang babaan ang halaga ng `vocab_size` at patakbuhin ang code sa ibaba, at tingnan kung paano ito nakakaapekto sa katumpakan. Dapat mong asahan ang kaunting pagbaba sa katumpakan, ngunit hindi masyadong malaki, kapalit ng mas mataas na pagganap.


## Pagsasanay ng BoW classifier

Ngayon na natutunan natin kung paano bumuo ng Bag-of-Words na representasyon ng ating teksto, magtuturo tayo ng isang classifier gamit ito. Una, kailangan nating i-convert ang ating dataset para sa pagsasanay sa paraang lahat ng positional vector representations ay ma-convert sa Bag-of-Words na representasyon. Magagawa ito sa pamamagitan ng pagpasa ng `bowify` function bilang `collate_fn` parameter sa standard torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Ngayon, mag-define tayo ng isang simpleng classifier neural network na naglalaman ng isang linear layer. Ang laki ng input vector ay katumbas ng `vocab_size`, at ang laki ng output ay tumutugma sa bilang ng mga klase (4). Dahil ang sinusolusyonan natin ay isang classification task, ang huling activation function ay `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Ngayon ay itatakda natin ang standard na PyTorch training loop. Dahil medyo malaki ang ating dataset, para sa layunin ng pagtuturo, magte-train tayo para sa isang epoch lamang, at minsan ay mas mababa pa sa isang epoch (ang pag-specify ng parameter na `epoch_size` ay nagbibigay-daan sa atin na limitahan ang training). Iuulat din natin ang naipong training accuracy habang nagte-training; ang dalas ng pag-uulat ay tinutukoy gamit ang parameter na `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams at N-Grams

Isang limitasyon ng bag of words na pamamaraan ay ang ilang mga salita ay bahagi ng mga multi-word na ekspresyon. Halimbawa, ang salitang 'hot dog' ay may ganap na ibang kahulugan kaysa sa mga salitang 'hot' at 'dog' sa ibang konteksto. Kung palaging ire-represent natin ang mga salitang 'hot' at 'dog' gamit ang parehong mga vector, maaaring malito ang ating modelo.

Upang matugunan ito, madalas ginagamit ang **N-gram representations** sa mga pamamaraan ng dokumento klasipikasyon, kung saan ang dalas ng bawat salita, bi-word, o tri-word ay isang kapaki-pakinabang na tampok para sa pag-train ng mga classifier. Sa bigram representation, halimbawa, idaragdag natin ang lahat ng pares ng salita sa vocabulary, bukod pa sa mga orihinal na salita.

Narito ang isang halimbawa kung paano gumawa ng bigram bag of word representation gamit ang Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Ang pangunahing kahinaan ng N-gram na pamamaraan ay ang mabilis na paglaki ng laki ng bokabularyo. Sa praktika, kailangan nating pagsamahin ang N-gram na representasyon sa ilang mga teknik sa pagbabawas ng dimensyon, tulad ng *embeddings*, na tatalakayin natin sa susunod na yunit.

Upang magamit ang N-gram na representasyon sa ating **AG News** dataset, kailangan nating bumuo ng espesyal na ngram na bokabularyo:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Maaari nating gamitin ang parehong code tulad ng nasa itaas upang sanayin ang classifier, ngunit magiging napaka-inefficient ito sa memorya. Sa susunod na unit, magsasanay tayo ng bigram classifier gamit ang embeddings.

> **Note:** Maaari mo lamang iwan ang mga ngrams na lumalabas sa teksto nang higit sa tinukoy na bilang ng beses. Sisiguraduhin nito na ang mga bihirang bigrams ay maaalis, at mababawasan nang malaki ang dimensionality. Upang gawin ito, itakda ang parameter na `min_freq` sa mas mataas na halaga, at obserbahan ang pagbabago sa haba ng vocabulary.


## Term Frequency Inverse Document Frequency TF-IDF

Sa BoW na representasyon, pantay ang timbang ng bawat paglitaw ng salita, anuman ang mismong salita. Gayunpaman, malinaw na ang mga madalas na salita, tulad ng *a*, *in*, atbp., ay mas hindi mahalaga para sa klasipikasyon kumpara sa mga espesyalisadong termino. Sa katunayan, sa karamihan ng mga gawain sa NLP, may mga salita na mas may kaugnayan kaysa sa iba.

Ang **TF-IDF** ay nangangahulugang **term frequency–inverse document frequency**. Isa itong bersyon ng bag of words, kung saan sa halip na binary na 0/1 na halaga na nagpapakita ng paglitaw ng isang salita sa isang dokumento, isang floating-point na halaga ang ginagamit, na may kaugnayan sa dalas ng paglitaw ng salita sa corpus.

Mas pormal, ang timbang na $w_{ij}$ ng isang salita $i$ sa dokumento $j$ ay tinutukoy bilang:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kung saan
* $tf_{ij}$ ay ang bilang ng paglitaw ng $i$ sa $j$, ibig sabihin, ang BoW na halaga na nakita natin dati
* $N$ ay ang bilang ng mga dokumento sa koleksyon
* $df_i$ ay ang bilang ng mga dokumento na naglalaman ng salita $i$ sa buong koleksyon

Ang halaga ng TF-IDF na $w_{ij}$ ay tumataas nang proporsyonal sa bilang ng beses na lumitaw ang isang salita sa isang dokumento at nababawasan batay sa bilang ng mga dokumento sa corpus na naglalaman ng salita, na tumutulong upang maiakma ang katotohanan na ang ilang mga salita ay mas madalas lumitaw kaysa sa iba. Halimbawa, kung ang salita ay lumitaw sa *bawat* dokumento sa koleksyon, $df_i=N$, at $w_{ij}=0$, at ang mga terminong iyon ay ganap na hindi papansinin.

Madali kang makakagawa ng TF-IDF vectorization ng teksto gamit ang Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Konklusyon

Gayunpaman, kahit na nagbibigay ang mga TF-IDF na representasyon ng timbang sa dalas ng iba't ibang salita, hindi nito kayang ipakita ang kahulugan o pagkakasunod-sunod. Tulad ng sinabi ng kilalang lingguwista na si J. R. Firth noong 1935, “Ang kumpletong kahulugan ng isang salita ay palaging nakabatay sa konteksto, at walang pag-aaral ng kahulugan na hiwalay sa konteksto ang maaaring ituring na seryoso.” Matututuhan natin sa mga susunod na bahagi ng kurso kung paano makuha ang impormasyong nakabatay sa konteksto mula sa teksto gamit ang pagmomodelo ng wika.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
